 # part 6: logistic regression - with coefficients and CIs

 Reviewers were generally interested in they way models our models work and the reasons for why they generated their predictions. They asked us to comment more on model machinery, and to report the contributions of input variables to model outcomes in more detail.

 In this notebook, we provide code to generate coefficients and confidence intervals for logistic regression models. We do this in a separate notebook for convenience and transparency, and and to prevent small changes in prior model results that we initially reported.

 we use the python statsmodels library.

In [0]:
import pandas as pd
import numpy as np
import python_modules.constants as constants
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score, auc
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

debug = False
set_intercept  = False
cat_vars = constants.CATEGORICAL_PRE
cat_ord  = constants.CATEGORICAL_ORDER
con_vars = constants.CONTINUOUS_PRE
ref_cats_dict = constants.REF_CATS



In [0]:
ref_cats_list = []
for keyname, valname in zip(ref_cats_dict.keys(), ref_cats_dict.values()):
    if valname.endswith('.'):
        ref_cats_list += [keyname]
ref_cats_list = ref_cats_list



In [0]:
ref_cats_list


 ## utils

 one hot encode features

In [0]:
def one_hot_encode(df):
    df_oh = df.copy()
    vs = []
    for v in cat_vars:
        df_oh[v] = df_oh[v].astype('str')
        vs += [v]
    # the easiest way to do this would be to set drop_first=True in the next line
    # however, we want to specify our own reference levels for categorical vars
    # therefore we have to specify which cols to drop
    df_oh = pd.get_dummies(df_oh, columns = vs, drop_first=False)

    df_oh.drop(columns=ref_cats_list, inplace=True)

    return df_oh


 score preds on auc

In [0]:
def generate_results_roc(y_test, y_score, plotting=False):
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    if plotting == True:
        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f' + str(roc_auc))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, .2])
        plt.ylim([0.0, .6])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic curve')
        plt.show()
    print('AUC: ' , roc_auc)


 ## leak

 load data

In [0]:
leak_train = pd.read_csv('study_data/LEAK_training.csv')
leak_train_original = pd.read_csv('study_data/LEAK_training_original.csv')
leak_val = pd.read_csv('study_data/LEAK_validation.csv')
leak_test = pd.read_csv('study_data/LEAK_testing.csv')

# drop unused col:
leak_train.drop(['Unnamed: 0'], axis=1, inplace=True)
leak_train_original.drop(['Unnamed: 0'], axis=1, inplace=True)
leak_val.drop(['Unnamed: 0'], axis=1, inplace=True)
leak_test.drop(['Unnamed: 0'], axis=1, inplace=True)



In [0]:
print(leak_train_original['RENAL_INSUFFICIENCY'].value_counts())
print(leak_val['RENAL_INSUFFICIENCY'].value_counts())
print(leak_test['RENAL_INSUFFICIENCY'].value_counts())


 we can explicitly confirm that we rebalanced data as intended in the previous notebook:

In [0]:
if debug == True:
    print(leak_train['LEAK'].value_counts())
    print(leak_train_original['LEAK'].value_counts())


 split data out into features and labels for each of training, testing, and validation sets

In [0]:
leak_train_labels = leak_train['LEAK']
leak_train_features = leak_train.drop('LEAK', axis=1)
leak_valid_labels = leak_val['LEAK']
leak_valid_features = leak_val.drop('LEAK', axis=1)
leak_testing_labels = leak_test['LEAK']
leak_testing_features = leak_test.drop('LEAK', axis=1)
leak_training_original_labels = leak_train_original['LEAK']
leak_training_original_features = leak_train_original.drop('LEAK', axis=1)


 one-hot encode categorical vars

In [0]:
leak_train_features_oh = one_hot_encode(leak_train_features)
leak_train_features_original_oh = one_hot_encode(leak_training_original_features)



In [0]:
leak_valid_features_oh = one_hot_encode(leak_valid_features)
leak_test_features_oh = one_hot_encode(leak_testing_features)



In [0]:
leak_train_features_original_oh.columns


 ### a note on intercepts

 if we want to set an intercept for models, can set an intercept col as follows. this results in worse AUCs by a little bit, and since we are after prediction more than interpretability,  don't do it.

In [0]:
if set_intercept == True:
    leak_train_features_oh['intercept'] = 1
    leak_valid_features_oh['intercept'] = 1
    leak_test_features_oh['intercept'] = 1


 ### back to modelling leak risk

 train models. use two different solvers and make sure results agree. of note, all signif coefficients match up between these solvers (not shown) and also matched up on a test of this package against SAS (also not shown).

In [0]:
logit_mod_bfgs = sm.Logit(leak_train_labels, leak_train_features_oh)
logit_res_bfgs = logit_mod_bfgs.fit(maxiter=500, method='bfgs')



In [0]:
logit_mod_ncg = sm.Logit(leak_train_labels, leak_train_features_oh)
logit_res_ncg = logit_mod_ncg.fit(maxiter=500, method='ncg')



In [0]:
preds_bfgs = logit_res_bfgs.predict(exog=leak_test_features_oh)
preds_ncg = logit_res_ncg.predict(exog=leak_test_features_oh)



In [0]:
generate_results_roc(leak_testing_labels, preds_bfgs)
generate_results_roc(leak_testing_labels, preds_ncg)


 we will report results using the ncg solver since it's done in the fewest steps and the results are essentially the same.

 show that models agree on which values are insignificant

In [0]:
print(list(logit_res_ncg.pvalues[logit_res_ncg.pvalues > 0.01].index))



In [0]:
print(list(logit_res_bfgs.pvalues[logit_res_bfgs.pvalues > 0.01].index))


 re-run models without insignificant columns to prove they aren't throwing the model off.

In [0]:
insig_cols_leak = list(logit_res_bfgs.pvalues[logit_res_bfgs.pvalues > 0.01].index)



In [0]:
leak_train_features_oh_sig = leak_train_features_oh.drop(columns=insig_cols_leak).copy()
leak_valid_features_oh_sig = leak_valid_features_oh.drop(columns=insig_cols_leak).copy()
leak_test_features_oh_sig  = leak_test_features_oh.drop(columns=insig_cols_leak).copy()



In [0]:
logit_mod_ncg_sig = sm.Logit(leak_train_labels, leak_train_features_oh_sig)
logit_res_ncg_sig = logit_mod_ncg_sig.fit(maxiter=500, method='ncg')



In [0]:
preds_ncg_sig = logit_res_ncg_sig.predict(exog=leak_test_features_oh_sig)


 the results are the same. we default to reporting the full model in order to maintain consistency with the machine learning model inputs.

In [0]:
generate_results_roc(leak_testing_labels, preds_ncg_sig)


 the coefficient for unknown IVC filter status is enormous. drop it to prove that it is not throwing the model off

In [0]:
leak_train_features_oh_sig_no_ivc0 = leak_train_features_oh_sig.drop(columns=['IVCF_0']).copy()
leak_valid_features_oh_sig_no_ivc0 = leak_valid_features_oh_sig.drop(columns=['IVCF_0']).copy()
leak_test_features_oh_sig_no_ivc0  = leak_test_features_oh_sig.drop(columns=['IVCF_0']).copy()



In [0]:
logit_mod_ncg_sig_no_ivc0 = sm.Logit(leak_train_labels, leak_train_features_oh_sig_no_ivc0)
logit_res_ncg_sig_no_ivc0 = logit_mod_ncg_sig_no_ivc0.fit(maxiter=500, method='ncg')



In [0]:
preds_ncg_sigsig_no_ivc0 = logit_res_ncg_sig_no_ivc0.predict(exog=leak_test_features_oh_sig_no_ivc0)



In [0]:
generate_results_roc(leak_testing_labels, preds_ncg_sigsig_no_ivc0)


 we can also show that grouping asa 5 with asa 4 does not improve model performance.

In [0]:
leak_train_features_oh_no_asa_5 = leak_train_features_oh.copy()
leak_valid_features_oh_no_asa_5 = leak_valid_features_oh.copy()
leak_test_features_oh_no_asa_5 = leak_test_features_oh.copy()

leak_train_features_oh_no_asa_5['ASACLASS_4'] = np.where(leak_train_features_oh_no_asa_5['ASACLASS_5']==1, 1, leak_train_features_oh_no_asa_5['ASACLASS_4'])
leak_valid_features_oh_no_asa_5['ASACLASS_4'] = np.where(leak_valid_features_oh_no_asa_5['ASACLASS_5']==1, 1, leak_valid_features_oh_no_asa_5['ASACLASS_4'])
leak_test_features_oh_no_asa_5['ASACLASS_4'] = np.where(leak_test_features_oh_no_asa_5['ASACLASS_5']==1, 1, leak_test_features_oh_no_asa_5['ASACLASS_4'])

leak_train_features_oh_no_asa_5.drop(columns='ASACLASS_5', inplace=True)
leak_valid_features_oh_no_asa_5.drop(columns='ASACLASS_5', inplace=True)
leak_test_features_oh_no_asa_5.drop(columns='ASACLASS_5', inplace=True)

logit_mod_ncg_asa45 = sm.Logit(leak_train_labels, leak_train_features_oh_no_asa_5)
logit_res_ncg_asa45 = logit_mod_ncg_asa45.fit(maxiter=500, method='ncg')

preds_ncg_sig_asa45 = logit_res_ncg_asa45.predict(exog=leak_test_features_oh_no_asa_5)

generate_results_roc(leak_testing_labels, preds_ncg_sig_asa45)


 and can show that the result holds if we drop insig cols again

In [0]:
print(list(logit_res_ncg_asa45.pvalues[logit_res_ncg.pvalues > 0.01].index))



In [0]:
insig_cols_leak_asa45 = list(logit_res_ncg_asa45.pvalues[logit_res_ncg_asa45.pvalues > 0.01].index)



In [0]:
leak_train_features_oh_no_asa_5_sig = leak_train_features_oh_no_asa_5.drop(columns=insig_cols_leak_asa45).copy()
leak_test_features_oh_no_asa5_sig  = leak_test_features_oh_no_asa_5.drop(columns=insig_cols_leak_asa45).copy()



In [0]:
logit_mod_ncg_sig_no_asa_5 = sm.Logit(leak_train_labels, leak_train_features_oh_no_asa_5_sig)
logit_res_ncg_sig_no_asa_5 = logit_mod_ncg_sig.fit(maxiter=500, method='ncg')



In [0]:
preds_ncg_sig_no_asa_5 = logit_res_ncg_sig_no_asa_5.predict(exog=leak_test_features_oh_no_asa5_sig)



In [0]:
generate_results_roc(leak_testing_labels, preds_ncg_sig_no_asa_5)



 ### in keeping with reporting guidelines, measure model performance on testing and validation data as well.

 training - with oversampling

In [0]:
preds_ncg_train = logit_res_ncg.predict(exog=leak_train_features_oh)



In [0]:
generate_results_roc(leak_train_labels, preds_ncg_train)


 training - without oversampling (this is what we report)

In [0]:
preds_ncg_train_original = logit_res_ncg.predict(exog=leak_train_features_original_oh)



In [0]:
generate_results_roc(leak_training_original_labels, preds_ncg_train_original)


 validation

In [0]:
preds_ncg_valid = logit_res_ncg.predict(exog=leak_valid_features_oh)



In [0]:
generate_results_roc(leak_valid_labels, preds_ncg_valid)


 ## clot

 I will not go through all the scenarios that i did above. the code can be easily adapted to do so. the same results hold in this context. here we just show results on testing as well as training and validation sets.

In [0]:
clot_train = pd.read_csv('study_data/CLOT_training.csv')
clot_train_original = pd.read_csv('study_data/CLOT_training_original.csv')
clot_val = pd.read_csv('study_data/CLOT_validation.csv')
clot_test = pd.read_csv('study_data/CLOT_testing.csv')

# drop unused col:
clot_train.drop(['Unnamed: 0'], axis=1, inplace=True)
clot_train_original.drop(['Unnamed: 0'], axis=1, inplace=True)
clot_val.drop(['Unnamed: 0'], axis=1, inplace=True)
clot_test.drop(['Unnamed: 0'], axis=1, inplace=True)


 split data out into features and labels for each of training, testing, and validation sets

In [0]:
clot_train_labels = clot_train['CLOT']
clot_train_features = clot_train.drop('CLOT', axis=1)
clot_valid_labels = clot_val['CLOT']
clot_valid_features = clot_val.drop('CLOT', axis=1)
clot_testing_labels = clot_test['CLOT']
clot_testing_features = clot_test.drop('CLOT', axis=1)
clot_training_original_labels = clot_train_original['CLOT']
clot_training_original_features = clot_train_original.drop('CLOT', axis=1)


 one-hot encoding

In [0]:
clot_train_features_oh = one_hot_encode(clot_train_features)
clot_valid_features_oh = one_hot_encode(clot_valid_features)
clot_test_features_oh = one_hot_encode(clot_testing_features)
clot_train_features_original_oh = one_hot_encode(clot_training_original_features)


 train models

In [0]:
logit_mod_ncg_clot = sm.Logit(clot_train_labels, clot_train_features_oh)
logit_res_ncg_clot = logit_mod_ncg_clot.fit(maxiter=500, method='ncg')


 predictions on testing data

In [0]:
preds_ncg_clot = logit_res_ncg_clot.predict(exog=clot_test_features_oh)



In [0]:
generate_results_roc(clot_testing_labels, preds_ncg_clot)


 training - oversampled

In [0]:
preds_ncg_clot_train = logit_res_ncg_clot.predict(exog=clot_train_features_oh)



In [0]:
generate_results_roc(clot_train_labels, preds_ncg_clot_train)


 training - not oversampled, report in paper

In [0]:
preds_ncg_clot_train_original = logit_res_ncg_clot.predict(exog=clot_train_features_original_oh)



In [0]:
generate_results_roc(clot_training_original_labels, preds_ncg_clot_train_original)


 validation

In [0]:
preds_ncg_clot_valid = logit_res_ncg_clot.predict(exog=clot_valid_features_oh)



In [0]:
generate_results_roc(clot_valid_labels, preds_ncg_clot_valid)


 ### save predictions

 leak

In [0]:
test_preds_leak = pd.DataFrame(preds_ncg)
test_preds_leak.rename(columns = {0:'lr_leak_test'}, inplace = True)
test_preds_leak.to_csv('results/study_models_LEAK/LEAK_logistic_preds_test')



In [0]:
valid_preds_leak = pd.DataFrame(preds_ncg_valid)
valid_preds_leak.rename(columns = {0:'lr_leak_valid'}, inplace = True)
valid_preds_leak.to_csv('results/study_models_LEAK/LEAK_logistic_preds_valid')



In [0]:
train_preds_leak = pd.DataFrame(preds_ncg_train_original)
train_preds_leak.rename(columns = {0:'lr_leak_train'}, inplace = True)
train_preds_leak.to_csv('results/study_models_LEAK/LEAK_logistic_preds_train')


 clot

In [0]:
test_preds_clot = pd.DataFrame(preds_ncg_clot)
test_preds_clot.rename(columns = {0:'lr_clot_test'}, inplace = True)
test_preds_clot.to_csv('results/study_models_CLOT/CLOT_logistic_preds_test')



In [0]:
valid_preds_clot = pd.DataFrame(preds_ncg_clot_valid)
valid_preds_clot.rename(columns = {0:'lr_clot_valid'}, inplace = True)
valid_preds_clot.to_csv('results/study_models_CLOT/CLOT_logistic_preds_valid')



In [0]:
train_preds_clot = pd.DataFrame(preds_ncg_clot_train_original)
train_preds_clot.rename(columns = {0:'lr_clot_train'}, inplace = True)
train_preds_clot.to_csv('results/study_models_CLOT/CLOT_logistic_preds_train')


 ## outputting tables

 get LR coefficients into the same format as the XGB var importance strings.

 handle it programatically. There are a few manipulation steps but worth it to avoid manual mapping and associated errors.

 Ideally, this would have been set up up-front.

In [0]:
#cat_ord
coeff_desc_dict = {}
coeff_rank_dict = {}



In [0]:
for key in cat_ord.keys():
    for i,j in enumerate(cat_ord[key]):
        coeff_string = key + '_' + str(i)
        descriptor_string = cat_ord[key][i]
        coeff_desc_dict[coeff_string] = descriptor_string



In [0]:
i = 0
for key in coeff_desc_dict.keys():
    coeff_rank_dict[key] = i
    i += 1



In [0]:
# from https://stackoverflow.com/questions/51734180/converting-statsmodels-summary-object-to-pandas-dataframe?rq=1
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })
    results_df.reset_index(drop=False, inplace=True)

    for i in ref_cats_list:
        results_df = results_df.append({'index':i} , ignore_index=True)
    
    results_df['odds_ratio'] = np.exp(results_df['coeff'])
    results_df['odds_ratio_lower'] = np.exp(results_df['conf_lower'])
    results_df['odds_ratio_higher'] = np.exp(results_df['conf_higher'])
    
    results_df = results_df.round(3)
    
    results_df['p_round'] = np.where(results_df['pvals'] >= 0.001, results_df['pvals'], '< 0.001')

    return results_df



In [0]:
r_logit_res_ncg = results_summary_to_dataframe(logit_res_ncg)



In [0]:
r_logit_res_ncg['desc'] = list(r_logit_res_ncg['index'].map(coeff_desc_dict))
r_logit_res_ncg['rank'] = list(r_logit_res_ncg['index'].map(coeff_rank_dict))
r_logit_res_ncg.sort_values(by='rank', inplace=True)



In [0]:
r_logit_res_clot_ncg = results_summary_to_dataframe(logit_res_ncg_clot)



In [0]:
r_logit_res_clot_ncg['desc'] = list(r_logit_res_clot_ncg['index'].map(coeff_desc_dict))
r_logit_res_clot_ncg['rank'] = list(r_logit_res_clot_ncg['index'].map(coeff_rank_dict))
r_logit_res_clot_ncg.sort_values(by='rank', inplace=True)



In [0]:
r_logit_res_ncg.to_csv('results/study_models_LEAK/lr_leak_coeffs.csv')



In [0]:
r_logit_res_clot_ncg.to_csv('results/study_models_CLOT/lr_clot_coeffs.csv')


 these tables are not fully formatted. the rest of the details are taken care of in ms office

In [0]:
r_logit_res_ncg['abs_or'] = 1 - abs(r_logit_res_ncg['odds_ratio'])
r_logit_res_ncg.sort_values(by='abs_or').head(20)



In [0]:
r_logit_res_clot_ncg['abs_or'] = 1 - abs(r_logit_res_ncg['odds_ratio'])
r_logit_res_clot_ncg.sort_values(by='abs_or').head(20)

